In [1]:
!pip install tensorflow

   ---------------------------------------- 0.0/390.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/390.2 MB ? eta -:--:--
   ---------------------------------------- 0.8/390.2 MB 2.2 MB/s eta 0:02:55
   ---------------------------------------- 1.3/390.2 MB 2.5 MB/s eta 0:02:37
   ---------------------------------------- 1.8/390.2 MB 2.6 MB/s eta 0:02:27
   ---------------------------------------- 2.6/390.2 MB 2.7 MB/s eta 0:02:22
   ---------------------------------------- 3.1/390.2 MB 2.8 MB/s eta 0:02:17
   ---------------------------------------- 3.9/390.2 MB 3.0 MB/s eta 0:02:10
   ---------------------------------------- 4.7/390.2 MB 3.1 MB/s eta 0:02:06
    --------------------------------------- 5.5/390.2 MB 3.2 MB/s eta 0:02:02
    --------------------------------------- 6.6/390.2 MB 3.3 MB/s eta 0:01:57
    --------------------------------------- 7.3/390.2 MB 3.4 MB/s eta 0:01:52
    --------------------------------------- 8.1/390.2 MB 3.4 MB/s eta 0:01:51


In [36]:
!pip install PIL

ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [14]:
!pip install scipy

   ---------------------------------------- 0.0/44.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/44.8 MB ? eta -:--:--
   ---------------------------------------- 0.5/44.8 MB 2.4 MB/s eta 0:00:19
   ---------------------------------------- 0.5/44.8 MB 2.4 MB/s eta 0:00:19
    --------------------------------------- 0.8/44.8 MB 745.8 kB/s eta 0:00:59
   - -------------------------------------- 1.3/44.8 MB 1.2 MB/s eta 0:00:38
   - -------------------------------------- 1.8/44.8 MB 1.5 MB/s eta 0:00:30
   -- ------------------------------------- 2.9/44.8 MB 1.9 MB/s eta 0:00:22
   --- ------------------------------------ 3.9/44.8 MB 2.3 MB/s eta 0:00:18
   ---- ----------------------------------- 5.2/44.8 MB 2.7 MB/s eta 0:00:15
   ----- ---------------------------------- 6.3/44.8 MB 3.0 MB/s eta 0:00:13
   ------ --------------------------------- 7.6/44.8 MB 3.3 MB/s eta 0:00:12
   ------- -------------------------------- 8.7/44.8 MB 3.4 MB/s eta 0:00:11
   --------

In [1]:
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Parameter
IMG_SIZE = (224, 224)  # Ukuran gambar untuk MobileNetV2
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 0.001

In [3]:
# Direktori dataset
train_dir = "train"
test_dir = "test"

In [4]:
# Augmentasi Data
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [5]:
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [6]:
# Generator Data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)


Found 6881 images belonging to 116 classes.


In [7]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

Found 1292 images belonging to 116 classes.


In [8]:
# Informasi jumlah kelas
num_classes = len(train_generator.class_indices)

In [9]:
# Model Transfer Learning
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Bekukan layer pada model dasar

In [10]:
# Tambahkan lapisan khusus
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Dropout untuk mencegah overfitting
x = Dense(128, activation="relu")(x)
predictions = Dense(num_classes, activation="softmax")(x)

In [11]:
# Buat model
model = Model(inputs=base_model.input, outputs=predictions)

In [12]:
# Kompilasi model
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [13]:
# Training
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator
)

c:\Users\MAHASISWA\Documents\GitHub\Nusantara_Culture_Database\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ImportError: This requires the scipy module. You can install it via `pip install scipy`